In [ ]:
import networkx as nx
import numpy as np
import itertools

def generate_regular_graph():
    # 这里简单以正则图为例, 鼓励同学们尝试在其他类型的图(具体可查看如下的nx文档)上测试算法性能
    # nx文档 https://networkx.org/documentation/stable/reference/generators.html
    graph = nx.random_graphs.random_regular_graph(d=99, n=200, seed=2023)
    return graph, len(graph.nodes), len(graph.edges)

def generate_erdos_renyi_graph():
    graph = nx.random_graphs.erdos_renyi_graph(n=180, p=0.52, seed=2023)
    return graph, len(graph.nodes), len(graph.edges)

data_path = 'regular'

if data_path == 'regular':
    graph, n_nodes, n_edges = generate_regular_graph()
elif data_path == 'ER':
    graph, n_nodes, n_edges = generate_erdos_renyi_graph()
    
print(n_nodes, n_edges)
def get_fitness(graph, x, threshold=0):
    # 获得Cuts值需要将图分为两部分, 这里默认以0为阈值把解分成两块.
    g1 = np.where(x == 0)[0]
    g2 = np.where(x == 1)[0]
    return -nx.cut_size(graph, g1, g2) / n_edges

import numpy as np
k = 8
n = n_nodes

In [ ]:
T = round(n*k*k*2*np.exp(1))
mse = []
N = 100
neigh_num = 5

# 生成权重向量
def gen_mean_vector(N, m):
    def _distribution_number(sum, m):
        if m == 1:
            return [[sum]]
        vectors = []
        for i in range(1, sum - (m - 1) + 1):
            right_vec = _distribution_number(sum - i, m - 1)
            vectors.extend([i] + item for item in right_vec)
        return vectors
    
    vectors = _distribution_number(N + m, m)
    vectors = (np.array(vectors) - 1) / N
    return vectors, len(vectors)

# 初始化距离
def init_distances(vectors):
    distances = np.zeros((popSize, popSize))
    for i, j in itertools.product(range(popSize), range(popSize)):
        distance = ((vectors[i] - vectors[j]) ** 2).sum()
        distances[i][j] = distance
    return distances

# 初始化种群
def initialize(popSize):
    population, neighbors = [], []
    for _ in range(popSize):
        while True:
            ind = np.random.choice((0, 1), p = (1 - k / n, k / n), size = n)
            if ind.sum() <= k:
                break

        population.append(ind)
    for _ in range(popSize):
        sort_arg = np.argsort(distances[_])
        neighbors.append([sort_arg[i] for i in range(neigh_num)])
    return population, neighbors

vectors, popSize = gen_mean_vector(N, 2)
distances = init_distances(vectors)
population, neighbors = initialize(popSize)

In [ ]:
from scipy.linalg import lstsq

def mutation(ind):
    individual = ind.copy()
    p = k / n
    m = np.random.choice((0,1), p=(1-p, p), size=n)
    flip = 1 - individual
    individual = np.where(m, flip, individual)
    return np.array(individual)

def crossover(individual_a, individual_b, n):
    l = len(individual_a)
    offspring_a = np.zeros((n))
    offspring_b = np.zeros((n))

    m = np.arange(l) < np.random.randint(l + 1)
    offspring_a = np.where(m, individual_a, individual_b)
    offspring_b = np.where(~m, individual_a, individual_b)

    return offspring_a, offspring_b

In [ ]:
function1_values, function2_values = [0 for _ in range(popSize)], [0 for _ in range(popSize)]
for i in range(popSize):
    loss = get_fitness(graph, population[i])
    function2_values[i] = population[i].sum() if (0 < population[i].sum() <= 2 * k) else 999999999999
    function1_values[i] = loss if (0 <= population[i].sum() <= 2 * k) else 999999999999

In [ ]:
import itertools
EP = []  # 前沿
EP_fx = []  # ep对应的目标值
mse = []

def funcs(ind):
    loss = get_fitness(graph, ind)
    f2 = ind.sum() if (0 < ind.sum() <= 2 * k) else 999999999999
    f1 = loss if (0 <= ind.sum() <= 2 * k) else 999999999999
    return f1, f2

for _ in range(T):
    for i in range(len(population)):
        neighbor = neighbors[i]
        idx = np.random.randint(0, neigh_num, size=2)
        p1, p2 = neighbor[idx[0]], neighbor[idx[1]] # 索引
        d1, d2 = crossover(population[p1], population[p2], n)
        d1, d2 = mutation(d1), mutation(d2)
        
        idx2 = np.random.randint(0, neigh_num, size=2)
        t1, t2 = neighbor[idx2[0]], neighbor[idx2[1]] # 索引
        
        d1_f1, d1_f2 = funcs(d1)
        d1_comp = vectors[t1][0] * (d1_f1 - function1_values[t1]) + \
            vectors[t1][1] * (d1_f2 - function2_values[t1]) / (2 * k) # d1和t1比较
        if d1_comp < 0:
            population[t1] = d1
            function1_values[t1], function2_values[t1] = d1_f1, d1_f2
            
            # 将新解与EP每一个进行比较，删除被新解支配的，如果新解没有被旧解支配，则保留
            accept_new = True
            for j in range(len(EP) - 1, -1, -1):  # 从后往前遍历
                old_fx = EP_fx[j]
                if (old_fx[0] < d1_f1 and old_fx[1] <= d1_f2) or (old_fx[0] <= d1_f1 and old_fx[1] < d1_f2):
                    accept_new = False
                    break
                elif (old_fx[0] > d1_f1 and old_fx[1] >= d1_f2) or (old_fx[0] >= d1_f1 and old_fx[1] > d1_f2)\
                    or (old_fx[0] == d1_f1 and old_fx[1] == d1_f2) / (2 * k):
                    del EP[j], EP_fx[j]
                    continue

            if accept_new:
                EP.append(d1)
                EP_fx.append([d1_f1, d1_f2])

        d2_f1, d2_f2 = funcs(d2)
        d2_comp = vectors[t2][0] * (d2_f1 - function1_values[t2]) + \
            vectors[t2][1] * (d2_f2 - function2_values[t2]) # d2和t2比较
        if d2_comp < 0:
            population[t2] = d2
            function1_values[t2], function2_values[t2] = d2_f1, d2_f2
            
            # 将新解与EP每一个进行比较，删除被新解支配的，如果新解没有被旧解支配，则保留
            accept_new = True
            for j in range(len(EP) - 1, -1, -1):  # 从后往前遍历
                old_fx = EP_fx[j]
                if (old_fx[0] < d2_f1 and old_fx[1] <= d2_f2) or (old_fx[0] <= d2_f1 and old_fx[1] < d2_f2):
                    accept_new = False
                    break
                elif (old_fx[0] > d2_f1 and old_fx[1] >= d2_f2) or (old_fx[0] >= d2_f1 and old_fx[1] > d2_f2)\
                    or (old_fx[0] == d2_f1 and old_fx[1] == d2_f2):
                    del EP[j], EP_fx[j]
                    continue

            if accept_new:
                EP.append(d2)
                EP_fx.append([d2_f1, d2_f2])
            
    if _ and _ % (T // 10000) == 0:
        res = np.array(EP_fx).T
        mse_loss = min(res[0][res[1] <= 8])
        mse.append(mse_loss)
        print(mse_loss, len(EP))

In [ ]:
from matplotlib import pyplot as plt
plt.scatter(res[0], res[1])

In [ ]:
plt.plot(mse)

In [ ]:
ep = res
tt = [_ * T // 100 for _ in range(100)]
mse = np.array(mse)


import matplotlib.pyplot as plt
plt.style.use(['science'])
with plt.style.context(['science']):
    plt.figure()
    plt.plot(tt, mse)
    plt.xlabel('epoch')
    plt.ylabel('mse')
    plt.title(f'moea: {data_path}')
    plt.savefig(f'MOEA_{data_path}.pdf')
    plt.close()
    
with plt.style.context(['science']):
    plt.scatter(ep[1], ep[0])
    plt.xlabel('k')
    plt.ylabel('mse')
    plt.title(f'Pareto front- moea: {data_path}')
    plt.savefig(f'MOEA_{data_path}_pareto.pdf', bbox_inches='tight')

In [ ]:
np.save(f'MOEA_{data_path}_t.npy', tt)
np.save(f'MOEA_{data_path}_mse.npy', mse)
np.save(f'MOEA_{data_path}_pareto.npy', ep)